# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f2f084f685-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the directors for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`| Quentin Tarantino      | node |






Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for director

3. Identify the BGP for film genre

4. Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

5. Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films). 

6. Identify the BGP for Academy Awards 

7. Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

8. Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

## Task 1
Identify the BGP for films

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [5]:
# query example
# let's find objects of node Quentin Tarantino
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3772 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q212026'), ('name', 'Mira Sorvino')]
[('obj', 'http://www.wikidata.org/entity/Q2566554'), ('name', '48th British Academy Film Awards')]
[('obj', 'http://www.wikidata.org/entity/Q833154'), ('name', 'Edgar Awards')]
[('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q113068'), ('name', 'Hungarian Order of Merit')]
[('obj', 'http://www.wikidata.org/entity/Q13452531'), ('name', 'Commandeur des Arts et des Lettres\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q17290934'), ('name', 'Lentapedia')]
[('obj', 'http://www.wikidata.org/entity/Q179808'), ('name', "Palme d'Or")]
[('obj', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'star on Hollywood Walk of Fame')]
[('obj', 'http://www.wikidat

In [6]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q188473 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3569'), ('name', 'Cultureel Woordenboek ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('

In [7]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film')]
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q188473 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('obj', 'http://www.wikidata.org/entity/Q1762165'), ('name', 'action fiction')]
2


In [9]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film')]
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
# FILTER by "film"
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q188473 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?name.
   FILTER(regex(?name,"film"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


In [12]:
# query example
# [('obj', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film')]
# [('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
# [('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
# FILTER by "film"
# reterive BGP for 'film'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q11424 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   #FILTER(regex(?name,"film"))
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p',

In [ ]:
# let's try to retrieve all the predicates and objs of film "What's Up, Tiger Lily?"
queryString = """
SELECT DISTINCT ?p ?pName ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 ?p ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P58'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('obj', 'http://www.wikidata.org/entity/Q25089'), ('objName', 'Woody Allen')]
[('p', 'http://www.wikidata.org/prop/direct/P750'), ('obj', 'http://www.wikidata.org/entity/Q465224'), ('objName', 'American International Pictures')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('obj', 'http://www.wikidata.org/entity/Q376138'), ('objName', 'Charles H. Joffe')]
[('p', 'http://www.wikidata.org/prop/direct/P344'), ('obj', 'http://www.wikidata.org/entity/Q6381680'), ('objName', 'Kazuo Yamada')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('obj', 'http://www.wikidata.org/entity/Q1053608'), ('objName', "The Lovin' Spoonful")]
[('p', 'http://www.wikidata.org/prop/direct/

Final query for this task

In [19]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q971865 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?objName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('objName', 'film')]
1


## Task 2
Identify the BGP for director

In [20]:
# write your queries
# query example
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3772 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q212026'), ('name', 'Mira Sorvino')]
[('obj', 'http://www.wikidata.org/entity/Q2566554'), ('name', '48th British Academy Film Awards')]
[('obj', 'http://www.wikidata.org/entity/Q833154'), ('name', 'Edgar Awards')]
[('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q113068'), ('name', 'Hungarian Order of Merit')]
[('obj', 'http://www.wikidata.org/entity/Q13452531'), ('name', 'Commandeur des Arts et des Lettres\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q17290934'), ('name', 'Lentapedia')]
[('obj', 'http://www.wikidata.org/entity/Q179808'), ('name', "Palme d'Or")]
[('obj', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'star on Hollywood Walk of Fame')]
[('obj', 'http://www.wikidat

In [21]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3455803 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5870903'), ('name', 'Category:Directors')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q483501'), ('name', 'artist')]
[('obj', 'http://www.wikidata.org/entity/Q2137554'), ('name', 'direction')]
[('obj', 'http://www.wikidata.org/entity/Q2462658'), ('name', 'manager')]
[('obj', 'http://www.wikidata.org/entity/Q2500638'), ('name', 'creator')]
[('obj', 'http://www.wikidata.org/entity/Q2607241'), ('name', 'mise en scène')]
[('obj', 'http://www.wikidata.org/entity/Q8104450'), ('name', 'Category:Wikipedia categories named after filmmakers')]
[('obj', 'http://www.wikidata.org/entity/P57'), ('name', 'director')]
[('obj', 'http://www.wikidata.org/entity/Q10809861'), ('name', 'Template:Infobox film director')]
10


In [22]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
# let's try to search predicates of director
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q3455803 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P918'), ('name', 'NOC Occupation Code')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2888'), ('name', 'exact match')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3553'), ('name', 'Zhihu topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p',

In [23]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3455803 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5870903'), ('name', 'Category:Directors')]
[('obj', 'http://www.wikidata.org/entity/Q28640'), ('name', 'profession')]
[('obj', 'http://www.wikidata.org/entity/Q483501'), ('name', 'artist')]
[('obj', 'http://www.wikidata.org/entity/Q2137554'), ('name', 'direction')]
[('obj', 'http://www.wikidata.org/entity/Q2462658'), ('name', 'manager')]
[('obj', 'http://www.wikidata.org/entity/Q2500638'), ('name', 'creator')]
[('obj', 'http://www.wikidata.org/entity/Q2607241'), ('name', 'mise en scène')]
[('obj', 'http://www.wikidata.org/entity/Q8104450'), ('name', 'Category:Wikipedia categories named after filmmakers')]
[('obj', 'http://www.wikidata.org/entity/P57'), ('name', 'director')]
[('obj', 'http://www.wikidata.org/entity/Q10809861'), ('name', 'Template:Infobox film director')]
10


In [24]:
# write your queries
# [('obj', 'http://www.wikidata.org/entity/Q3455803'), ('name', 'director')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q971865 wdt:P57 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q25089'), ('name', 'Woody Allen')]
[('obj', 'http://www.wikidata.org/entity/Q27351'), ('name', 'Senkichi Taniguchi')]
2


In [25]:
# write your queries
# try to find predicate that link with film "What's Up, Tiger Lily? " to Senkichi Taniguchi and Woody Allen
queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
   # bind something
   wd:Q971865 ?p wd:Q27351 ;
        ?p wd:Q25089 .
   # get the label
   ?p sc:name ?pName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P57'), ('pName', 'director')]
1


In [2]:
# let's try with subclass of film director
# filter by 'director'
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q2526255 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?objName.
   FILTER REGEX (?objName, "director")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


Final query for this task

In [29]:
# Final query
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
   # bind something
   wd:Q2526255 wdt:P279 ?obj .
   # get the label
   ?obj sc:name ?objName.
   FILTER REGEX (?objName, "director")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q3455803'), ('objName', 'director')]
1


## Task 3
Identify the BGP for film genre

In [32]:
# write your queries
# find properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1003'), ('name', 'National Library of Romania ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('name', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [33]:
# write your queries
# find objects of director Woody Allen 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q25089 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q349357'), ('name', 'Woody Herman')]
[('obj', 'http://www.wikidata.org/entity/Q1140914'), ('name', 'Soon-Yi Previn')]
[('obj', 'http://www.wikidata.org/entity/Q22673707'), ('name', 'Bechet Dumaine Allen')]
[('obj', 'http://www.wikidata.org/entity/Q22673708'), ('name', 'Manzie Tio Allen')]
[('obj', 'http://www.wikidata.org/entity/Q18426'), ('name', 'The Bronx')]
[('obj', 'http://www.wikidata.org/entity/Q102642'), ('name', 'Diane Keaton')]
[('obj', 'http://www.wikidata.org/entity/Q3404523'), ('name', 'David di Donatello for Best Foreign Actor')]
[('obj', 'http://www.wikidata.org/entity/Q599316'), ('name', 'New School')]
[('obj', 'http://www.wikidata.org/entity/Q19961034'), ('name', 'Woody')]
[('obj', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris')]
[('obj', 'http://www.wikidata.org/entity/Q3831197'), ('name', 'Letty Aronson')]
[('obj', 'http://www.wikidata.org/entity/Q31579'), ('name', 'David di Donatello')]
[('obj

In [34]:
# write your queries
# find objects of director Woody Allen
# [('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q971865 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q25089'), ('name', 'Woody Allen')]
[('obj', 'http://www.wikidata.org/entity/Q465224'), ('name', 'American International Pictures')]
[('obj', 'http://www.wikidata.org/entity/Q376138'), ('name', 'Charles H. Joffe')]
[('obj', 'http://www.wikidata.org/entity/Q6381680'), ('name', 'Kazuo Yamada')]
[('obj', 'http://www.wikidata.org/entity/Q1053608'), ('name', "The Lovin' Spoonful")]
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('obj', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan')]
[('obj', 'http://www.wikidata.org/entity/Q1860'), ('name', 'English')]
[('obj', 'http://www.wikidata.org/entity/Q22006653'), ('name', 'color')]
[('obj', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q723685'), ('name', 'video on demand')]
[('obj', 'http://www.wikidata.org/entity/Q907311')

In [35]:
# write your queries
# find properties of film 'What's Up, Tiger Lily?'
# [('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q971865 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('name', 'Netflix ID')]
[('p', '

In [37]:
# write your queries
# find properties of Q971865
# [('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
# [('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q971865 wdt:P136 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]
1


In [40]:
# write your queries
# find properties of Q971865
# [('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
# [('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q971865 wdt:P136 ?genre .
   ?genre wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


Final query for this task

In [41]:
# # write your final query
# find properties of Q971865
# [('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
# [('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
# [('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q971865 wdt:P136 ?genre .
   ?genre wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


## Task 4
Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

In [2]:
# write your queries
#[('p', 'http://www.wikidata.org/prop/direct/P27'), ('name', 'country of citizenship')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'), ('name', 'continent')]
#[('obj', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?films ?sName ?country ?name
WHERE {
   # bind something
   ?films wdt:P31 wd:Q5;
           wdt:P136 wd:Q172980;
           wdt:P27 ?country.
           
   ?country wdt:P31 wd:Q6256; 
           wdt:P30 wd:Q46.        
           
   # get the label
   ?films sc:name ?sName .
   ?country sc:name ?name .
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('films', 'http://www.wikidata.org/entity/Q309240'), ('sName', 'Jean Giraud'), ('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France')]
1


In [4]:
# write your queries
# film genre (Q201658)
queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q201658 .
   # get the label
   ?s sc:name ?sName.
   FILTER REGEX(?sName, "western")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q212781'), ('sName', 'spaghetti western')]
[('s', 'http://www.wikidata.org/entity/Q4836991'), ('sName', 'B western')]
[('s', 'http://www.wikidata.org/entity/Q83646243'), ('sName', 'western animation')]
[('s', 'http://www.wikidata.org/entity/Q78528852'), ('sName', 'neo-western')]
4


In [6]:
# write your queries
# film genre (Q201658)
queryString = """
SELECT DISTINCT ?genre ?genreName
WHERE {
   # bind something
   ?s wdt:P136 ?genre .
   # get the label
   ?genre sc:name ?genreName
   FILTER REGEX(?genreName, ".*Western.*")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q26263265'), ('genreName', 'Western art')]
[('genre', 'http://www.wikidata.org/entity/Q4235011'), ('genreName', 'Space Western')]
[('genre', 'http://www.wikidata.org/entity/Q9730'), ('genreName', 'Western classical music')]
[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genreName', 'Western film')]
[('genre', 'http://www.wikidata.org/entity/Q491158'), ('genreName', 'Revisionist Western')]
[('genre', 'http://www.wikidata.org/entity/Q12197074'), ('genreName', 'Western novel')]
[('genre', 'http://www.wikidata.org/entity/Q367591'), ('genreName', 'Western literature')]
[('genre', 'http://www.wikidata.org/entity/Q7439349'), ('genreName', 'Scrumpy and Western')]
[('genre', 'http://www.wikidata.org/entity/Q21590660'), ('genreName', 'Western')]
[('genre', 'http://www.wikidata.org/entity/Q1425111'), ('genreName', 'Western music (country)')]
[('genre', 'http://www.wikidata.org/entity/Q3756540'), ('genreName', 'Western painting')]
[('genr

In [9]:
# write your queries
# film genre (Q201658)
# [('genre', 'http://www.wikidata.org/entity/Q172980'), ('genreName', 'Western film')]
queryString = """
SELECT DISTINCT ?film ?filmName
WHERE {
   # bind something
   ?film wdt:P136 wd:Q172980;
       wdt:P31 wd:Q11424.
       
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q15882802'), ('filmName', 'The Storm')]
[('film', 'http://www.wikidata.org/entity/Q16200305'), ('filmName', 'Fangs of the Arctic')]
[('film', 'http://www.wikidata.org/entity/Q17011616'), ('filmName', 'Mr. Texas')]
[('film', 'http://www.wikidata.org/entity/Q17036710'), ('filmName', 'The Drifting Avenger')]
[('film', 'http://www.wikidata.org/entity/Q17052173'), ('filmName', 'The Desert of the Lost')]
[('film', 'http://www.wikidata.org/entity/Q17053125'), ('filmName', 'Between Dangers')]
[('film', 'http://www.wikidata.org/entity/Q7569044'), ('filmName', 'South of Santa Fe')]
[('film', 'http://www.wikidata.org/entity/Q17184091'), ('filmName', 'With Buffalo Bill on the U. P. Trail')]
[('film', 'http://www.wikidata.org/entity/Q17190227'), ('filmName', 'Easy Going')]
[('film', 'http://www.wikidata.org/entity/Q17639337'), ('filmName', 'Skedaddle Gold')]
[('film', 'http://www.wikidata.org/entity/Q17639444'), ('filmName', 'Speedy Spurs')]
[('film

Final query for this task

In [ ]:
# write your final query

## Task 5
Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films).

In [57]:
# write your queries
# director (P57)
# film (Q11424)
# Woody Allen (Q25089)
# let's search all the films whom director is Woody Allen
queryString = """
SELECT DISTINCT ?film ?filmName
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
       wdt:P31 wd:Q11424 .
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?")]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('filmName', 'Broadway Danny Rose')]
[('film', 'http://www.wikidata.org/entity/Q733677'), ('filmName', 'Match Point')]
[('film', 'http://www.wikidata.org/entity/Q740143'), ('filmName', 'Husbands and Wives')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('filmName', 'Bananas')]
[('film', 'http://www.wikidata.org/entity/Q1321004'), ('filmName', 'You Will Meet a Tall Dark Stranger')]
[('film'

In [59]:
# write your queries
# director (P57)
# film (Q11424)
# Woody Allen (Q25089)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
queryString = """
SELECT DISTINCT ?film ?filmName ?year
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?year.
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City'), ('year', '2001-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('year', '2011-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('year', '2011-08-18T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('year', '1990-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('year', '1991-02-28T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?"), ('year', '1966-01-01T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('year', '1994-09-16T00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('year', '1995-04-20T00:00:00Z')]
[('film', 'h

In [62]:
# write your queries
# director (P57)
# film (Q11424)
# Woody Allen (Q25089)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
queryString = """
SELECT DISTINCT ?film ?filmName (YEAR(?date) AS ?publicationYear)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City'), ('publicationYear', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('publicationYear', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('publicationYear', '1990')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('publicationYear', '1991')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?"), ('publicationYear', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('publicationYear', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('publicationYear', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('publicationYear', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1354109')

In [67]:
# write your queries
# director (P57)
# film (Q11424)
# publication date (P577)
# Woody Allen (Q25089)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
# let's try to find with first date of publication because some are showing with double
queryString = """
SELECT DISTINCT ?film ?filmName (YEAR(?date) AS ?publicationYear)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   
   filter NOT EXISTS{
       ?film wdt:P577 ?secondDate .
       filter(?secondDate < ?date) .
   }
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City'), ('publicationYear', '2001')]
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('publicationYear', '2011')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('publicationYear', '1990')]
[('film', 'http://www.wikidata.org/entity/Q971865'), ('filmName', "What's Up, Tiger Lily?"), ('publicationYear', '1966')]
[('film', 'http://www.wikidata.org/entity/Q1004531'), ('filmName', 'Bullets Over Broadway'), ('publicationYear', '1994')]
[('film', 'http://www.wikidata.org/entity/Q1354109'), ('filmName', 'Mighty Aphrodite'), ('publicationYear', '1995')]
[('film', 'http://www.wikidata.org/entity/Q729026'), ('filmName', 'Broadway Danny Rose'), ('publicationYear', '1984')]
[('film', 'http://www.wikidata.org/entity/Q733677'), ('filmName', 'Match Point'), ('publicationYear', '2005')]
[('film', 'http://www.wikidata.org/entity/Q740143

In [68]:
# write your queries
# director (P57)
# film (Q11424)
# publication date (P577)
# Woody Allen (Q25089)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
# let's try to find with first date of publication because some are showing with double publication dates
queryString = """
SELECT DISTINCT ?film ?filmName (YEAR(?date) AS ?publicationYear) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   
   filter NOT EXISTS{
       ?film wdt:P577 ?secondDate .
       filter(?secondDate < ?date) .
   }
   # get the label
   ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q206124'), ('filmName', 'Midnight in Paris'), ('publicationYear', '2011'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q806092'), ('filmName', 'Bananas'), ('publicationYear', '1971'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q187423'), ('filmName', 'Zelig'), ('publicationYear', '1983'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q3986392'), ('filmName', 'The Concert for New York City'), ('publicationYear', '2001'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q682262'), ('filmName', 'Alice'), ('publicationYear', '1990'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q614009'), ('filmName', 'Anything Else'), ('publicationYear', '2003'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q910523'), ('filmName', 'Shadows and Fog'), ('publicationYear', '1991'), ('numFilms', '1')]
[('film', 'http://www.wikidata.org/entity/Q1133277'), ('filmName', 

In [5]:
# write your queries
# director (P57)
# film (Q11424)
# publication date (P577)
# Woody Allen (Q25089)
# Quentin Tarantino (Q3772)
# let's search all the films whom director is Woody Allen and count it
# let's find publication year of each film
# let's try to find with first date of publication because some are showing with double publication dates
queryString = """
SELECT DISTINCT (YEAR(?date) AS ?publicationYear) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 wd:Q25089;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   
   filter NOT EXISTS{
       ?film wdt:P577 ?secondDate .
       filter(?secondDate < ?date) .
   }
   # get the label
  # ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('publicationYear', '2000'), ('numFilms', '1')]
[('publicationYear', '2004'), ('numFilms', '1')]
[('publicationYear', '2007'), ('numFilms', '1')]
[('publicationYear', '1985'), ('numFilms', '1')]
[('publicationYear', '1978'), ('numFilms', '1')]
[('publicationYear', '1966'), ('numFilms', '1')]
[('publicationYear', '1999'), ('numFilms', '1')]
[('publicationYear', '1998'), ('numFilms', '1')]
[('publicationYear', '1996'), ('numFilms', '1')]
[('publicationYear', '1971'), ('numFilms', '1')]
[('publicationYear', '2006'), ('numFilms', '1')]
[('publicationYear', '1987'), ('numFilms', '2')]
[('publicationYear', '1990'), ('numFilms', '1')]
[('publicationYear', '2010'), ('numFilms', '1')]
[('publicationYear', '2017'), ('numFilms', '1')]
[('publicationYear', '1973'), ('numFilms', '1')]
[('publicationYear', '1991'), ('numFilms', '1')]
[('publicationYear', '2015'), ('numFilms', '1')]
[('publicationYear', '2002'), ('numFilms', '1')]
[('publicationYear', '1983'), ('numFilms', '1')]
[('publicati

In [5]:
# write your queries
# director (P57)
# film (Q11424)
# publication date (P577)
# Woody Allen (Q25089)
# Quentin Tarantino (Q3772)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
# let's try to find with first date of publication because some are showing with double
queryString = """
SELECT DISTINCT ?dir ?dirName (YEAR(?date) AS ?publicationYear) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 ?dir;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   ?dir sc:name ?dirName
   filter NOT EXISTS{
       ?film wdt:P577 ?secondDate .
       filter(?secondDate < ?date) .
   }
   # get the label
  # ?film sc:name ?filmName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('dir', 'http://www.wikidata.org/entity/Q3318647'), ('dirName', 'Mohamed Khan'), ('publicationYear', '1984'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q744326'), ('dirName', 'Victor Saville'), ('publicationYear', '1933'), ('numFilms', '3')]
[('dir', 'http://www.wikidata.org/entity/Q543563'), ('dirName', 'Phillip Noyce'), ('publicationYear', '2014'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q15930508'), ('dirName', 'Henry Chan'), ('publicationYear', '2013'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q750983'), ('dirName', 'Riccardo Freda'), ('publicationYear', '1951'), ('numFilms', '3')]
[('dir', 'http://www.wikidata.org/entity/Q3101789'), ('dirName', 'George Nicholls, Jr.'), ('publicationYear', '1938'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q11985796'), ('dirName', 'F. Šestka'), ('publicationYear', '1935'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q6235036'), ('dirName', 'Poul Wela

In [9]:
# write your queries
# director (P57)
# film (Q11424)
# publication date (P577)
# Woody Allen (Q25089)
# Quentin Tarantino (Q3772)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
# let's try to find with first date of publication because some are showing with double
# we need to Consider only films directed by Woody Allen or Quentin Tarantino
queryString = """
SELECT DISTINCT ?dir ?dirName (YEAR(?date) AS ?publicationYear) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 ?dir;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   ?dir sc:name ?dirName
   FILTER(?dir=wd:Q25089 || ?dir=wd:Q3772)
   filter NOT EXISTS{
       ?film wdt:P577 ?secondDate .
       filter(?secondDate < ?date) .
   }
   # get the label
  # ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('publicationYear', '1994'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('publicationYear', '2012'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '2001'), ('numFilms', '2')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '1966'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '2000'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '2016'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '1982'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('publica

Final query for this task

In [9]:
# # write your final query
# director (P57)
# film (Q11424)
# publication date (P577)
# Woody Allen (Q25089)
# Quentin Tarantino (Q3772)
# let's search all the films whom director is Woody Allen
# let's find publication year of each film
# let's try to find with first date of publication because some are showing with double
# we need to Consider only films directed by Woody Allen or Quentin Tarantino
queryString = """
SELECT DISTINCT ?dir ?dirName (YEAR(?date) AS ?publicationYear) (COUNT(?film) AS ?numFilms)
WHERE {
   # bind something
   ?film wdt:P57 ?dir;
       wdt:P31 wd:Q11424 ;
       wdt:P577 ?date.
   ?dir sc:name ?dirName
   FILTER(?dir=wd:Q25089 || ?dir=wd:Q3772)
   filter NOT EXISTS{
       ?film wdt:P577 ?secondDate .
       filter(?secondDate < ?date) .
   }
   # get the label
  # ?film sc:name ?filmName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('publicationYear', '1994'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('publicationYear', '2012'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '2001'), ('numFilms', '2')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '1966'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '2000'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '2016'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q25089'), ('dirName', 'Woody Allen'), ('publicationYear', '1982'), ('numFilms', '1')]
[('dir', 'http://www.wikidata.org/entity/Q3772'), ('dirName', 'Quentin Tarantino'), ('publica

## Task 6
Identify the BGP for Academy Awards

In [48]:
# write your queries
# let's look to first query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3772 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q212026'), ('name', 'Mira Sorvino')]
[('obj', 'http://www.wikidata.org/entity/Q2566554'), ('name', '48th British Academy Film Awards')]
[('obj', 'http://www.wikidata.org/entity/Q833154'), ('name', 'Edgar Awards')]
[('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q10800557'), ('name', 'film actor')]
[('obj', 'http://www.wikidata.org/entity/Q113068'), ('name', 'Hungarian Order of Merit')]
[('obj', 'http://www.wikidata.org/entity/Q13452531'), ('name', 'Commandeur des Arts et des Lettres\u200e')]
[('obj', 'http://www.wikidata.org/entity/Q17290934'), ('name', 'Lentapedia')]
[('obj', 'http://www.wikidata.org/entity/Q179808'), ('name', "Palme d'Or")]
[('obj', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'star on Hollywood Walk of Fame')]
[('obj', 'http://www.wikidat

In [52]:
# write your queries
# let's look to first query
# [('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q103360 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q96474679'), ('name', 'award for best direction')]
[('obj', 'http://www.wikidata.org/entity/Q160726'), ('name', 'Ang Lee')]
[('obj', 'http://www.wikidata.org/entity/Q55215'), ('name', 'Alejandro González Iñárritu')]
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q28028'), ('name', 'Alfonso Cuarón')]
[('obj', 'http://www.wikidata.org/entity/Q212329'), ('name', 'Academy of Motion Picture Arts and Sciences')]
[('obj', 'http://www.wikidata.org/entity/Q7146100'), ('name', 'Category:Best Directing Academy Award winners')]
[('obj', 'http://www.wikidata.org/entity/Q6667698'), ('name', 'Template:Academy Award for Best Director')]
[('obj', 'http://www.wikidata.org/entity/Q495980'), ('name', 'Bong Joon-ho')]
10


In [55]:
# write your queries
# let's look to first query
# [('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q103360 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   filter(regex(?name, "Academy Awards"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
1


Final query for this task

In [55]:
# write your queries
# # write your final query
# [('obj', 'http://www.wikidata.org/entity/Q103360'), ('name', 'Academy Award for Best Director')]
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q103360 ?p ?obj .
   # get the label
   ?obj sc:name ?name.
   filter(regex(?name, "Academy Awards"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
1


## Task 7
Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

In [4]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# award received (P166)
# director (P57)
# let's find academy awards won by films and also find director name
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName ?academyAward ?academyAwardName
WHERE {
    ?academyAward wdt:P31 wd:Q19020 .
    ?film wdt:P31 wd:Q11424;
        wdt:P57 ?director ;
        wdt:P166 ?academyAward .
        
    ?director sc:name ?directorName .
    ?film sc:name ?filmName .
    ?academyAward sc:name ?academyAwardName .
    filter (?director=wd:Q25089 || ?director=wd:Q3772)
}
LIMIT 100

"""
print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q153723'), ('filmName', 'Inglourious Basterds'), ('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('director', 'http://www.wikidata.org/entity/Q3772

In [6]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# award received (P166)
# director (P57)
# let's find academy awards won by films and also find director name
#group by director and film as well as count number of awards
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) as ?totalAwards)
WHERE {
    ?academyAward wdt:P31 wd:Q19020 .
    ?film wdt:P31 wd:Q11424;
        wdt:P57 ?director ;
        wdt:P166 ?academyAward .
        
    ?director sc:name ?directorName .
    ?film sc:name ?filmName .
    ?academyAward sc:name ?academyAwardName .
    filter (?director=wd:Q25089 || ?director=wd:Q3772)
}
group by ?film ?filmName ?director ?directorName
LIMIT 50

"""
print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q47300912'), ('filmName', 'Once Upon a Time in Hollywood'), ('totalAwards', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q845057'), ('filmName', 'Hannah and Her Sisters'), ('totalAwards', '3')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q153723'), ('filmName', 'Inglourious Basterds'), ('totalAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q104123'), ('filmName', 'Pulp Fiction'), ('totalAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmName', 'Django U

In [9]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# award received (P166)
# director (P57)
# let's find academy awards won by films and also find director name
#group by director and film as well as count number of awards
# lets try to get maximum academy awards so we need MAX fun
queryString = """
SELECT DISTINCT ?director ?directorName (MAX(?totalAwards) as ?maxNoOfAwards)
WHERE{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) as ?totalAwards)
WHERE {
    ?academyAward wdt:P31 wd:Q19020 .
    ?film wdt:P31 wd:Q11424;
        wdt:P57 ?director ;
        wdt:P166 ?academyAward .
        
    ?director sc:name ?directorName .
    ?film sc:name ?filmName .
    ?academyAward sc:name ?academyAwardName .
    filter (?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
group by ?film ?filmName ?director ?directorName
LIMIT 50

"""
print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxNoOfAwards', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxNoOfAwards', '3')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxNoOfAwards', '1')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxNoOfAwards', '4')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxNoOfAwards', '1')]
5


In [12]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# award received (P166)
# director (P57)
# let's find academy awards won by films and also find director name
#group by director and film as well as count number of awards
# lets try to get maximum academy awards so we need MAX fun
queryString = """
SELECT DISTINCT ?director ?directorName (MAX(?totalAwards) as ?maxNoOfAwards)
WHERE{
SELECT DISTINCT ?director ?directorName ?film ?filmName (COUNT(?academyAward) as ?totalAwards)
WHERE {
    ?academyAward wdt:P31 wd:Q19020 .
    ?film wdt:P31 wd:Q11424;
        wdt:P57 ?director ;
        wdt:P166 ?academyAward .
        
    ?director sc:name ?directorName .
    ?film sc:name ?filmName .
    ?academyAward sc:name ?academyAwardName .
    filter (?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
group by ?director ?directorName
LIMIT 50

"""
print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxNoOfAwards', '2')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxNoOfAwards', '4')]
2


In [45]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# award received (P166)
# director (P57)
# let's find academy awards won by films and also find director name
#group by director and film as well as count number of awards
# lets try to get maximum academy awards so we need MAX fun
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (MAX(?totalAwards) as ?maxNoOfAwards)
WHERE{
SELECT DISTINCT ?director ?directorName (COUNT(?academyAward) as ?totalAwards)
WHERE {
    ?academyAward wdt:P31 wd:Q19020 .
    ?film wdt:P31 wd:Q11424;
        wdt:P57 ?director ;
        wdt:P166 ?academyAward .
        
    ?director sc:name ?directorName .
    ?film sc:name ?filmName .
    #?academyAward sc:name ?academyAwardName .
    filter (?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
group by ?director ?directorName ?film ?filmName
LIMIT 50

"""
print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxNoOfAwards', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxNoOfAwards', '2')]
2


Final query for this task

In [3]:
# write your final query
# Academy Awards (Q19020)
# film (Q11424)
# award received (P166)
# director (P57)
# let's find academy awards won by films and also find director name
#group by director and film as well as count number of awards
# lets try to get maximum academy awards so we need MAX fun
queryString = """
SELECT DISTINCT ?director ?directorName ?film ?filmName (MAX(?totalAwards) as ?maxNoOfAwards)
WHERE{
SELECT DISTINCT ?director ?directorName (COUNT(?academyAward) as ?totalAwards)
WHERE {
    ?academyAward wdt:P31 wd:Q19020 .
    ?film wdt:P31 wd:Q11424;
        wdt:P57 ?director ;
        wdt:P166 ?academyAward .
        
    ?director sc:name ?directorName .
    ?film sc:name ?filmName .
    #?academyAward sc:name ?academyAwardName .
    filter (?director=wd:Q25089 || ?director=wd:Q3772)
}
GROUP BY ?film ?filmName ?director ?directorName
}
group by ?director ?directorName ?film ?filmName
LIMIT 50

"""
print("Results")
x=run_query(queryString)

Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorName', 'Woody Allen'), ('maxNoOfAwards', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorName', 'Quentin Tarantino'), ('maxNoOfAwards', '2')]
2


## Task 8
Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

In [48]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# nominated for (P1411)
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName
WHERE {
?academyAward wdt:P31 wd:Q19020 .
?film wdt:P31 wd:Q11424;
    wdt:P57 ?director;
    wdt:P1411 ?academyAward.
   
?director sc:name ?directorName .
?academyAward sc:name ?academyAwardName
FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q102427'), ('academyAwardName', 'Academy Award for Best Picture')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director')]
[('academyAward', 'http://www.wikidata.org/entity/Q106301'), ('academyAwardName', 'Academy Award for Best Supporting Actress')]
[('academyAward', 'http://www.wikidata.org/entity/Q281939'), ('academyAwardName', 'Academy Award for Best Film Editing')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original Screenplay')]
[('academyAward', 'http://www.wikidata.org/entity/Q103916'), ('academyAwardName', 'Academy Award for Best Actor')]
[('academyAward', 'http://www.wikidata.org/entity/Q106291'), ('academyAwardName', 'Academy Award for Best Supporting Actor')]
[('academyAward', 'http://www.wikidata.org/entity/Q131520'), ('academyAwardName', 'Academy Award for Best Cinematography')]
[('ac

In [50]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# nominated for (P1411) count its nomination for academy awards
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName (COUNT(?film) AS ?nominations)
WHERE {
?academyAward wdt:P31 wd:Q19020 .
?film wdt:P31 wd:Q11424;
    wdt:P57 ?director;
    wdt:P1411 ?academyAward.
   
?director sc:name ?directorName .
?academyAward sc:name ?academyAwardName
FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q277751'), ('academyAwardName', 'Academy Award for Best Production Design'), ('nominations', '5')]
[('academyAward', 'http://www.wikidata.org/entity/Q281939'), ('academyAwardName', 'Academy Award for Best Film Editing'), ('nominations', '3')]
[('academyAward', 'http://www.wikidata.org/entity/Q488651'), ('academyAwardName', 'Academy Award for Best Original Score'), ('nominations', '1')]
[('academyAward', 'http://www.wikidata.org/entity/Q103618'), ('academyAwardName', 'Academy Award for Best Actress'), ('nominations', '3')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director'), ('nominations', '9')]
[('academyAward', 'http://www.wikidata.org/entity/Q106301'), ('academyAwardName', 'Academy Award for Best Supporting Actress'), ('nominations', '11')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original 

In [58]:
# write your queries
# Academy Awards (Q19020)
# film (Q11424)
# nominated for (P1411) count its nomination for academy awards
# i was trying to find academy awards
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName (COUNT(?film) AS ?nominations)
WHERE {
?academyAward wdt:P31 wd:Q19020 .
?film wdt:P31 wd:Q11424;
    wdt:P57 ?director;
    wdt:P1411 ?academyAward.
   
?director sc:name ?directorName .
?academyAward sc:name ?academyAwardName
FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
#filter (regex(?name, "Academy Awards"))
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q277751'), ('academyAwardName', 'Academy Award for Best Production Design'), ('nominations', '5')]
[('academyAward', 'http://www.wikidata.org/entity/Q281939'), ('academyAwardName', 'Academy Award for Best Film Editing'), ('nominations', '3')]
[('academyAward', 'http://www.wikidata.org/entity/Q488651'), ('academyAwardName', 'Academy Award for Best Original Score'), ('nominations', '1')]
[('academyAward', 'http://www.wikidata.org/entity/Q103618'), ('academyAwardName', 'Academy Award for Best Actress'), ('nominations', '3')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director'), ('nominations', '9')]
[('academyAward', 'http://www.wikidata.org/entity/Q106301'), ('academyAwardName', 'Academy Award for Best Supporting Actress'), ('nominations', '11')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original 

Final query for this task

Final query for this task

In [59]:
# write your final query
# Academy Awards (Q19020)
# film (Q11424)
# nominated for (P1411) count its nomination for academy awards
queryString = """
SELECT DISTINCT ?academyAward ?academyAwardName (COUNT(?film) AS ?nominations)
WHERE {
?academyAward wdt:P31 wd:Q19020 .
?film wdt:P31 wd:Q11424;
    wdt:P57 ?director;
    wdt:P1411 ?academyAward.
   
?director sc:name ?directorName .
?academyAward sc:name ?academyAwardName
FILTER(?director=wd:Q25089 || ?director=wd:Q3772)
#filter (regex(?name, "Academy Awards"))
}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('academyAward', 'http://www.wikidata.org/entity/Q277751'), ('academyAwardName', 'Academy Award for Best Production Design'), ('nominations', '5')]
[('academyAward', 'http://www.wikidata.org/entity/Q281939'), ('academyAwardName', 'Academy Award for Best Film Editing'), ('nominations', '3')]
[('academyAward', 'http://www.wikidata.org/entity/Q488651'), ('academyAwardName', 'Academy Award for Best Original Score'), ('nominations', '1')]
[('academyAward', 'http://www.wikidata.org/entity/Q103618'), ('academyAwardName', 'Academy Award for Best Actress'), ('nominations', '3')]
[('academyAward', 'http://www.wikidata.org/entity/Q103360'), ('academyAwardName', 'Academy Award for Best Director'), ('nominations', '9')]
[('academyAward', 'http://www.wikidata.org/entity/Q106301'), ('academyAwardName', 'Academy Award for Best Supporting Actress'), ('nominations', '11')]
[('academyAward', 'http://www.wikidata.org/entity/Q41417'), ('academyAwardName', 'Academy Award for Best Writing, Original 